<a href="https://colab.research.google.com/github/aheiX/Teaching/blob/main/Transportation%20Problem%20(with%20emission%20reduction%20strategies).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transportation Problem with emission reduction strategies

## Required packages

In [74]:
# Load required pyhton packages
!pip install pulp
import pulp
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

## Data

In [80]:
# Supply node
S = ['Hamburg', 'Berlin', 'Munich']

# Supply
s = {'Hamburg': 100, 'Berlin': 200, 'Munich': 70}

# Demand nodes
D = ['Bremen', 'Cologne', 'Dresden']

# Demand
d = {'Bremen': 80, 'Cologne': 40, 'Dresden': 90}

# Transport costs
c = {
    'Hamburg': {'Bremen': 4, 'Cologne': 3, 'Dresden': 8},
    'Berlin': {'Bremen': 6, 'Cologne': 8, 'Dresden': 9},
    'Munich': {'Bremen': 7, 'Cologne': 6, 'Dresden': 5}
}

# Transport emissions
e = {'Hamburg': {'Bremen': 8, 'Cologne': 7, 'Dresden': 6},
     'Berlin': {'Bremen': 6, 'Cologne': 9, 'Dresden': 7},
     'Munich': {'Bremen': 4, 'Cologne': 3, 'Dresden': 9}}

## PuLP Model

In [137]:
def model_all_inclusive(S, s, D, d, c, e,
                        carbon_tax=None,
                        carbon_limit=None,
                        p_buy=None, p_sell=None, initial_amount=None,
                        obj=None):
  # Model
  model = pulp.LpProblem(name='Transport_Problem_Min_Emissions',
                        sense=pulp.constants.LpMinimize)

  # Decision variables
  x = pulp.LpVariable.dicts(name='x', indices=(S, D), lowBound=0, cat='Integer')

  # Dummy decision variables for reporting
  costs = pulp.LpVariable('costs', lowBound=0, upBound=None, cat=pulp.LpContinuous)
  emissions = pulp.LpVariable('emissions', lowBound=0, upBound=None, cat=pulp.LpContinuous)

  # Decision variables from carbon trading scenario
  if initial_amount is not None:
    buy = pulp.LpVariable('buy', lowBound=0, upBound=None, cat=pulp.LpContinuous)
    sell = pulp.LpVariable('sell', lowBound=0, upBound=None, cat=pulp.LpContinuous)

  # (1) Objective
  if obj == 'emissions':
    model += emissions, '(1) emissions'
  else:
      model += costs, '(1) costs'

  # add dummy decision variables to the model
  if carbon_tax is not None:
    # carbon tax
    model += costs == pulp.lpSum(c[i][j] * x[i][j] for i in S for j in D) + emissions*carbon_tax
  elif initial_amount is not None:
    # carbon trading
    model += costs == pulp.lpSum(c[i][j] * x[i][j] for i in S for j in D) + p_buy * buy - p_sell * sell
  else:
    model += costs == pulp.lpSum(c[i][j] * x[i][j] for i in S for j in D)

  model += emissions == pulp.lpSum(e[i][j] * x[i][j] for i in S for j in D)

  # (2)
  for i in S:
    model += s[i] >= pulp.lpSum(x[i][j] for j in D), '(2)_' + str(i)

  # (2)
  for j in D:
    model += d[j] == pulp.lpSum(x[i][j] for i in S), '(3)_' + str(j)

  # carbon limit
  if carbon_limit is not None:
    model += carbon_limit >= pulp.lpSum(x[i][j]*e[i][j] for i in S for j in D), 'Carbon_Limit'

  # carbon trading
  if initial_amount is not None:
    model += emissions <= initial_amount + buy - sell, 'Carbon_Trade'

  return model

## Solving

In [145]:
def solve_and_print(model, extended=True):
  model.solve()

  if extended:
    # get status
    print("Status:", pulp.LpStatus[model.status])

    # get objective value
    print('Objective value:', round(pulp.value(model.objective), 2))

    print("All variables:")
    for v in model.variables():
      if v.varValue > 0:
        print(v.name, "=", v.varValue)
  else:
    for v in model.variables():
      if v.varValue > 0 and v.name in ['costs', 'emissions']:
        print(v.name, "=", v.varValue)


def solve_and_store(model):
  model.solve()

  if pulp.LpStatus[model.status] == 'Optimal':

    for v in model.variables():
      if v.name == 'costs':
        opt_costs = v.varValue

      elif v.name == 'emissions':
        opt_emissions = v.varValue

    return opt_costs, opt_emissions

  else:

    # simplified_model = model_all_inclusive(S=S, s=s, D=D, d=d, c=c, e=e, obj='costs')
    # return solve_and_store(simplified_model)

    return None, None



## Experiments

In [191]:
def make_figure(x, y, x_name, y_name,
                x2, y2, y2_name,
                title):

  # Create figure with secondary y-axis
  fig = make_subplots(specs=[[{"secondary_y": True}]])

  # Add traces
  fig.add_trace(
      go.Scatter(x=x, y=y, name=y_name, line=dict(color='red')),
      secondary_y=False,
  )

  fig.add_trace(
      go.Scatter(x=x, y=[1010 for i in x], name='min costs', line=dict(color='red', dash='dot')),
      secondary_y=False
  )

  fig.add_trace(
      go.Scatter(x=x2, y=y2, name=y2_name, line=dict(color='green')),
      secondary_y=True,
  )

  fig.add_trace(
      go.Scatter(x=x2, y=[1080 for i in x2], name='min emissions', line=dict(color='green', dash='dash')),
      secondary_y=True
  )

  # Add figure title
  fig.update_layout(
      title_text=title,
      width=750,
      height=500,
      template='simple_white'
  )

  # Set x-axis title
  fig.update_xaxes(title_text=x_name)
#   fig.update_layout(
#     xaxis = dict(
#         tickmode = 'linear',
#         tick0 = 0.5,
#         dtick = 0.75
#     )
# )

  # Set y-axes titles
  fig.update_yaxes(title_text=y_name, secondary_y=False) #  tickmode='linear', tick0=500, dtick=100,
  fig.update_yaxes(title_text=y2_name, secondary_y=True)

  fig.show()


In [193]:
print('min costs')
model = model_all_inclusive(S=S, s=s, D=D, d=d, c=c, e=e, obj='costs')
solve_and_print(model, extended=False)

print('')
print('min emissions')
model = model_all_inclusive(S=S, s=s, D=D, d=d, c=c, e=e, obj='emissions')
solve_and_print(model, extended=False)

# carbon tax
carbon_taxes = [round(i, 1) for i in np.arange(0, 1.6, 0.01)]
costs = []
emissions = []
for carbon_tax in carbon_taxes:
  model = model_all_inclusive(S=S, s=s, D=D, d=d, c=c, e=e, obj='costs',
                              carbon_tax=carbon_tax)
  tmp_c, tmp_e = solve_and_store(model)
  costs.append(tmp_c)
  emissions.append(tmp_e)

make_figure(x=carbon_taxes, y=costs, x_name='carbon tax', y_name='costs',
            x2=carbon_taxes, y2=emissions, y2_name='emissions',
            title='Costs and emissions for a varying carbon tax')


# carbon limit
carbon_limits = [round(i, 1) for i in np.arange(1000, 1800, 10)]
costs = []
emissions = []
for carbon_limit in carbon_limits:
  model = model_all_inclusive(S=S, s=s, D=D, d=d, c=c, e=e, obj='costs',
                              carbon_limit=int(carbon_limit))
  tmp_c, tmp_e = solve_and_store(model)
  costs.append(tmp_c)
  emissions.append(tmp_e)

make_figure(x=carbon_limits, y=costs, x_name='carbon limit', y_name='costs',
            x2=carbon_limits, y2=emissions, y2_name='emissions',
            title='Costs and emissions for a varying carbon limits')


# carbon trading
initial_amounts = [round(i, 1) for i in np.arange(800, 2500, 20)]
p_buys = [1]
p_sells = [round(i, 1) for i in np.arange(0, 1, 0.1)]
p_sells = [0, 0.5, 1]
costs = []
emissions = []

for p_buy in p_buys:
  for p_sell in p_sells:
    costs = []
    emissions = []
    for initial_amount in initial_amounts:
      model = model_all_inclusive(S=S, s=s, D=D, d=d, c=c, e=e, obj='costs',
                                  p_buy=p_buy, p_sell=p_sell, initial_amount=initial_amount)

      # solve_and_print(model, extended=True)
      tmp_c, tmp_e = solve_and_store(model)
      costs.append(tmp_c)
      emissions.append(tmp_e)

    make_figure(x=initial_amounts, y=costs, x_name='initial carbon amount', y_name='costs',
                x2=initial_amounts, y2=emissions, y2_name='emissions',
                title='Costs and emissions for a varying initial carbon amounts (p_buy=' + str(p_buy) + ', p_sell=' + str(p_sell) + ')')

# print('')
# print('min costs with carbon trading' )

# solve_and_print(model, extended=False)


min costs
costs = 1010.0
emissions = 1650.0

min emissions
costs = 1470.0
emissions = 1080.0
